# Virtual diffractometer axis

Demonstrate a diffractometer where one of the expected axes must be computed
from one or more additional diffractometer positioners.

One case might be where a rotational axis $a$ is operated by a tangent linear
translation $y$ at fixed distance $x$ from the center of rotation.  In this
simple sketch, the relationships are defined: {math}`\tan{a} = y / x`

![sketch](../_static/th_tth-virt-axis-sketch.png)

## Preparation

The virtual axis must be defined with the features common to the other
diffractometer axes (all based on `ophyd.PositionerBase`).

Since this is a virtual *real* axis and not a *pseudo*, the easiest way to
ensure these features are available is to create a subclass of
`ophyd.SoftPositioner` (based on `ophyd.PositionerBase`) and override the parts
which are custom to the virtual axis.  A virtual *pseudo* axis should subclass
from `hklpy2.Hklpy2PseudoAxis`.

The `ophyd.DerivedSignal` is not based on `ophyd.PositionerBase` and, as such,
is missing many features of the positioner interface.

**Parameters**

- $x$ : A constant specified when creating the diffractometer object.  Same
  engineering units as $y$.
- $y$ : The physical translation axis.  Specify the *name* of this diffractometer
  Component when creating the diffractometer object.  The virtual positioner
  class will access the named diffractometer Component.
- $a$ : The virtual rotation angle.  When $a$ is read, its position is computed
  from  $y$.  Any movement of $a$ is translated into a movement of $y$.

### Virtual positioner class

We'll do this in two steps.  First, create a `VirtualPositioner` class which
could be used as a base class for any virtual axis.  The, use it as a base class
to create a `VirtualRotationPositioner` for the geometry of the `tth` axis of
this diffractometer.

Custom class for the particular specifications of this virtual axis. Describes
the **Parameters** above in the `__init__()` constructor method and implements
their design in the class **Overrides** and **Additions** methods.

axis type | base class
--- | ---
*pseudo* | `hklpy2.Hklpy2PseudoAxis`
*real* | `ophyd.SoftPositioner`

**Overrides**

Custom methods that *replace* base class methods.

- `_setup_move()`: Moves physical axis when virtual axis is moved.

**Additions**

Custom methods for this class, not provided by the base class.

The virtual axis limits are computed from the translation axis limits using the
`forward()` method.

- `forward()`: Math to compute virtual rotation from physical translation.
- `inverse()`: Math to compute physical translation from virtual rotation.
- `_finish_setup()`: Complete the axis setup after diffractometer is built.
- `_recompute_limits()`: Using `forward()`, compute limits of virtual axis from physical axis.
- `_cb_update_position()`: Called when physical position is changed.

In [1]:
import math
import time

from ophyd import Component
from ophyd import SoftPositioner, EpicsMotor, PVPositioner

class VirtualPositioner(SoftPositioner):
    """Base class for a diffractometer's virtual axis."""

    def __init__(self, *, physical_name: str = "", **kwargs):
        """Constructor.
        
        Subclass should override and add any additional kwargs, as needed.
        """
        if len(physical_name) == 0:
            raise ValueError("Must provide a value for 'physical_name'.")

        self._setup_finished: bool = False

        super().__init__(**kwargs)

        self.physical = getattr(self.parent, physical_name)

    def _setup_move(self, position, status):
        """Move requested to position."""
        self._finish_setup()
        self._run_subs(sub_type=self.SUB_START, timestamp=time.time())

        self._started_moving = True
        self._moving = False

        if self._setup_finished:
            self.physical._setup_move(self.inverse(position), status)

        self._set_position(position)
        self._done_moving()

    def forward(self, physical: float) -> float:
        """
        Return virtual position from physical position.
        
        Subclass should override.
        """
        return physical  # Subclass should redefine.

    def inverse(self, virtual: float) -> float:
        """
        Return physical position from virtual position.
        
        Subclass should override.
        """
        return virtual  # Subclass should redefine.

    def _cb_update_position(self, value, **kwargs):
        """Called when physical position is changed."""
        self._finish_setup()
        self._position = self.forward(value)

        # Update our position in diffractometer's internal cache.
        self.parent._real_cur_pos[self] = self._position

    def _finish_setup(self):
        """
        Complete the axis setup after diffractometer is built.
        
        Update our:
        
        * Position by subscription to readback changes.
        * Limits from physical axis.
        """
        if not self.connected or self._setup_finished:
            return

        try:
            physical = self.physical
        except AttributeError:
            return  # During initialization.

        # Readback signal is in different locations.
        if isinstance(physical, SoftPositioner):
            # Includes PseudoPositioner subclass
            readback = physical
        elif isinstance(physical, EpicsMotor):
            readback = physical.user_readback
        elif isinstance(physical, PVPositioner):
            readback = physical.readback
        else:
            raise TypeError(f"Unknown 'readback' for {physical.name}.")

        self._setup_finished = True
        readback.subscribe(self._cb_update_position)
        self._recompute_limits()

    def _recompute_limits(self) -> None:
        """Compute virtual axis limits from physical axis."""
        if self._setup_finished:
            self._limits = tuple(sorted(map(self.forward, self.physical.limits)))


Using `VirtualPositioner` as a base class, create the custom class for our virtual rotation axis.  Here, we override these methods:

- `__init__()` : Add additional `distance keyword argument.
- `forward()` : Math to compute virtual rotation from physical translation.
- `inverse()` : Math to compute physical translation from virtual rotation.

In [2]:
class VirtualRotationPositioner(VirtualPositioner):
    """Compute virtual rotation from physical translation axis."""

    def __init__(self, *, distance: float = 100, **kwargs):
        """Distance from translation axis zero position to center of rotation."""
        self.distance = distance  # same units as physical
        super().__init__(**kwargs)

    def forward(self, translation: float) -> float:
        """Return virtual rotation angle (degrees) from physical translation."""
        return math.atan2(translation, self.distance) * 180 / math.pi

    def inverse(self, rotation: float) -> float:
        """Return physical translation from virtual rotation angle (degrees)."""
        return self.distance * math.tan(rotation * math.pi / 180)


### Custom diffractometer class

Here, we pick the
[4-circle](https://blueskyproject.io/hklpy2/diffractometers.html#solver-hkl-soleil-geometry-e4cv)
`E4CV` geometry to demonstrate a diffractometer that uses this new
{class}`VirtualRotationPositioner` class.

- `tth` (as $a$): Override the existing `tth` Component with the
  `VirtualRotationPositioner`, supplying the additional kwargs.

- `dy` (as $y$): Add a `dy` Component as a `SoftPositioner`, supplying initial
  position and limits.

- `distance` (as $x$):  Pass this constant as a keyword argument when constructing
  the diffractometer object.

Note the `kind="hinted"` kwarg, which designates a Component to be included in a
live table or plot during a scan.

<details>
<summary>Why not use <tt>hklpy2.creator()</tt>?</summary>

We must write our own Python class. The `hklpy2.creator()` is not prepared
[yet](https://github.com/bluesky/hklpy2/issues/113) to create a diffractometer
with a custom class and keyword arguments such as this one.  

<details>
<summary>Maybe, in the future ...</summary>

```python
gonio = hklpy2.creator(
    name="gonio",
    solver="hkl_soleil",
    geometry="E4CV",
    reals = {
        omega: None,
        chi: None,
        phi: None,
        tth: {
            class: VirtualRotationPositioner,
            init_pos=0,
            t_name="dy",
            distance=1000,
            kind="hinted",
        },
        dy: {
            limits=(-10, 200),
            # all other kwargs use defaults
        }
    }
)
```

</details>

</details>
<br />

Using the factory function to create a base class, we define a custom class:

In [3]:
import hklpy2

MyBase = hklpy2.diffractometer_class_factory(
    solver="hkl_soleil",
    geometry="E4CV",
)


class MyGoniometer(MyBase):
    # Replace existing axis with virtual 2Theta.
    # Point it at the 'dy' axis (below).
    tth = Component(
        VirtualRotationPositioner,
        init_pos=0,
        physical_name="dy",
        distance=1000,
        kind="hinted",
    )

    # Add the translation axis 'dy'.
    dy = Component(
        SoftPositioner,
        init_pos=0,
        limits=(-10, 200),
        kind="hinted",
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.tth._finish_setup()

### Diffractometer object

- Move `dy` (physical axis), report `tth` and `l` at each step.
- Move `tth` (virtual axis), report `dy` and `l` at each step.
- Move `l` (pseudo axis), report `tth` and `dy` at each step.

In [4]:
gonio = MyGoniometer(name="gonio")
gonio.add_sample("vibranium", 2*3.14)
gonio.wh()

wavelength=1.0
pseudos: h=0, k=0, l=0
reals: omega=0, chi=0, phi=0, tth=0
positioners: dy=0


## Bluesky scans

Demonstrate the relationships between `dy`, `tth`, and `l` through bluesky scans.

- Scan `dy` (physical axis), report `tth` and `l` at each step.
- Scan `tth` (virtual axis), report `dy` and `l` at each step.
- Scan `l` (pseudo axis), report `tth` and `dy` at each step.

Start with a simple setup, but no detectors or data collection.  Just tables and plotting.

In [5]:
import bluesky
from bluesky import plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
bec.disable_plots()
RE = bluesky.RunEngine()
RE.subscribe(bec)

0

### Scan *physical* axis `dy`

In [6]:
RE(bp.scan([gonio.dy, gonio.tth, gonio.l], gonio.dy, -1, 10, 11))
gonio.wh()



Transient Scan ID: 1     Time: 2025-07-15 15:16:33
Persistent Unique Scan ID: 'f09d025e-3e13-4800-a267-8d29fd726f5c'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |   gonio_dy |  gonio_tth |    gonio_l |
+-----------+------------+------------+------------+------------+
|         1 | 15:16:33.7 |     -1.000 |     -0.057 |     -0.006 |
|         2 | 15:16:33.7 |      0.100 |      0.006 |      0.001 |
|         3 | 15:16:33.7 |      1.200 |      0.069 |      0.008 |
|         4 | 15:16:33.7 |      2.300 |      0.132 |      0.014 |
|         5 | 15:16:33.7 |      3.400 |      0.195 |      0.021 |
|         6 | 15:16:33.7 |      4.500 |      0.258 |      0.028 |
|         7 | 15:16:33.7 |      5.600 |      0.321 |      0.035 |
|         8 | 15:16:33.7 |      6.700 |      0.384 |      0.042 |
|         9 | 15:16:33.7 |      7.800 |      0.447 |      0.049 |
|        10 | 15:16:33.7 |      8.900 |      0.510 |      0.056 |
|

### Scan *virtual* axis `tth`

In [7]:
RE(bp.scan([gonio.dy, gonio.tth, gonio.l], gonio.tth, -0.1, 0.5, 11))
gonio.wh()



Transient Scan ID: 2     Time: 2025-07-15 15:16:33
Persistent Unique Scan ID: 'a21651c7-2f0d-4d86-88a5-9fac5a85c840'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |  gonio_tth |    gonio_l |   gonio_dy |
+-----------+------------+------------+------------+------------+
|         1 | 15:16:33.9 |     -0.100 |     -0.011 |     -1.745 |
|         2 | 15:16:33.9 |     -0.040 |     -0.004 |     -0.698 |
|         3 | 15:16:33.9 |      0.020 |      0.002 |      0.349 |
|         4 | 15:16:33.9 |      0.080 |      0.009 |      1.396 |
|         5 | 15:16:33.9 |      0.140 |      0.015 |      2.443 |
|         6 | 15:16:33.9 |      0.200 |      0.022 |      3.491 |
|         7 | 15:16:33.9 |      0.260 |      0.028 |      4.538 |
|         8 | 15:16:33.9 |      0.320 |      0.035 |      5.585 |
|         9 | 15:16:33.9 |      0.380 |      0.042 |      6.632 |
|        10 | 15:16:33.9 |      0.440 |      0.048 |      7.680 |
|

### Scan *pseudo* axis `l`

In [8]:
RE(bp.scan([gonio.dy, gonio.tth, gonio.l], gonio.l, 0.01, 0.05, 11))
gonio.wh()



Transient Scan ID: 3     Time: 2025-07-15 15:16:34
Persistent Unique Scan ID: 'c495ee40-8dc1-48c9-a4a2-29675b2bc6e8'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |    gonio_l |  gonio_tth |   gonio_dy |
+-----------+------------+------------+------------+------------+
|         1 | 15:16:34.1 |      0.010 |      0.091 |      1.593 |
|         2 | 15:16:34.1 |      0.014 |      0.128 |      2.230 |
|         3 | 15:16:34.1 |      0.018 |      0.164 |      2.867 |
|         4 | 15:16:34.1 |      0.022 |      0.201 |      3.503 |
|         5 | 15:16:34.1 |      0.026 |      0.237 |      4.140 |
|         6 | 15:16:34.1 |      0.030 |      0.274 |      4.777 |
|         7 | 15:16:34.1 |      0.034 |      0.310 |      5.414 |
|         8 | 15:16:34.1 |      0.038 |      0.347 |      6.051 |
|         9 | 15:16:34.1 |      0.042 |      0.383 |      6.688 |
|        10 | 15:16:34.1 |      0.046 |      0.420 |      7.325 |
|